In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## KNN : baseline
 *En principio usando un bootstrap del set de entrenamiento para medir accuracy. Se utilizo un 25% del set para entrenar y el 75% restante para la predicción. Se utilizó la representación TF-IDF para el entrenamiento ya que KNN funciona con valores numericos.*
 
Methods

* fit(self, X, y) Fit the model using X as training data and y as target values

* get_params(self[, deep]) Get parameters for this estimator.

* kneighbors(self[, X, n_neighbors, …]) Finds the K-neighbors of a point.

* kneighbors_graph(self[, X, n_neighbors, mode]) Computes the (weighted) graph of k-Neighbors for points in X

* predict(self, X) Predict the class labels for the provided data.

* predict_proba(self, X) Return probability estimates for the test data X.

* score(self, X, y[, sample_weight]) Return the mean accuracy on the given test data and labels.

* set_params(self, \*\*params) Set the parameters of this estimator.

### Basic features

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train, y_train)
max = knn.score(x_test, y_test)
actual = max
optimo = 1

for i in range(2, 100):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(x_train, y_train)
    actual = knn.score(x_test, y_test)
    if max < actual:
        optimo = i 
        max = actual

print(optimo, max)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=43)
all_accuracies = cross_val_score(estimator=knn, X=x_train, y=y_train, cv=40)
print(all_accuracies.mean())

### TF-IDF

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(train_vectors, y_text_train)
max = knn.score(test_vectors, y_text_test)
actual = max
optimo = 1

for i in range(2, 100):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(train_vectors, y_text_train)
    actual = knn.score(test_vectors, y_text_test)
    if max < actual:
        optimo = i 
        max = actual

print(optimo, max) #0.7961269499731038 con 49

### KNN con CountVectorizer

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(count_train, y_text_train)
max = knn.score(count_test, y_text_test)
actual = max
optimo = 1

for i in range(2, 100):
    knn = KNeighborsClassifier(n_neighbors = i)
    knn.fit(count_train, y_text_train)
    actual = knn.score(count_test, y_text_test)
    if max < actual:
        optimo = i 
        max = actual

print(optimo, max) #0.6778749159381304 con 1 (basura)

## XGBoost

### Basic features

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

#acc = 0.7305002689618074(objective ='binary:logistic', colsample_bytree = 0.6, learning_rate = 0.01, max_depth = 35,alpha = 0.5, n_estimators = 140) 
xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.6, learning_rate = 0.005,
                max_depth = 35,alpha = 0.5, n_estimators = 140) 
xg_reg.fit(x_train,y_train) 
y_pred = xg_reg.predict(x_test)

for i in range (0, len(y_pred)):
    if y_pred[i] >= 0.5:       
        y_pred[i] = 1 
    else:  
        y_pred[i]=0
        
accuracy = accuracy_score(y_pred,y_test)
print(accuracy)

In [ ]:
kfold = KFold(n_splits=10)
results = cross_val_score(xg_reg, x_train, y_train, cv=kfold)
results

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [12, 7]
xgb.plot_importance(xg_reg)

### TF-IDF

Busqueda de parametros

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(learning_rate = np.arange(0.001,0.1,0.005),
                              n_estimators = np.arange(15,300,15),
                              scale_pos_weight = np.arange(2,6,1),
                              max_depth = np.arange(15,40,2),min_child_weight= np.arange(1,10,1),
                              gamma = np.arange(0,1,0.1), alpha= np.arange(0.1,1,0.1),
                              subsample = np.arange(0,1,0.1), colsample_bytree = np.arange(0.5,0.8,0.05),
                              colsample_bylevel = np.arange(0.6,0.91,0.05))
xgb_model = xgb.XGBClassifier()

grid = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=20, n_jobs=1)
grid_result = grid.fit(train_vectors,y_text_train)
params_xgb_tfidf = grid_result.best_params_
print("Best parameters: ", params_xgb_tfidf)

In [ ]:
xg_reg = xgb.XGBClassifier(**params_xgb_tfidf)
xg_reg.fit(train_vectors,y_text_train)
preds = xg_reg.predict(test_vectors)
for i in range (0, len(preds)):
    if preds[i] >= 0.5:       
        preds[i] = 1 
    else:  
        preds[i] = 0

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(preds,y_text_test)
print(accuracy)

### CountVectorizer

Busqueda de parametros

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = dict(learning_rate = np.arange(0.01,0.5,0.02),
                              n_estimators = np.arange(15,300,15),
                              scale_pos_weight = np.arange(2,6,1),
                              max_depth = np.arange(15,40,2),min_child_weight= np.arange(1,10,1),
                              gamma = np.arange(0,0.5,0.1), alpha= np.arange(0.1,1,0.1),
                              subsample = np.arange(0.6,1,0.1), colsample_bytree = np.arange(0.5,0.8,0.05),
                              colsample_bylevel = np.arange(0.6,0.91,0.05))
classifier = xgb.XGBClassifier()

grid = RandomizedSearchCV(estimator=classifier, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=20, n_jobs=1)
grid_result = grid.fit(count_train,y_text_train)
params_xgb_count = grid_result.best_params_
print("Best parameters: ", params_xgb_count)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
# Mejores parametros encontrados
# (objective ='reg:logistic', colsample_bytree = 0.45, colsample_bylevel = 0.4, learning_rate = 0.05, max_depth = 25, min_child_weight = 1.1, alpha = 0.5, gamma =  0.4, n_estimators = 210)
xg_reg = xgb.XGBRegressor(**params_xgb_count)
xg_reg.fit(count_train,y_train) 
y_pred = xg_reg.predict(count_test)

for i in range (0, len(y_pred)):
    if y_pred[i] >= 0.5:       
        y_pred[i] = 1 
    else:  
        y_pred[i]=0
        
accuracy = accuracy_score(y_pred,y_test)
print(accuracy)

### Hashing vectorizer

In [ ]:
import xgboost as xgb

xg_reg = xgb.XGBRegressor(objective ='reg:logistic', 
                colsample_bytree = 0.45, colsample_bylevel = 0.4, learning_rate = 0.05,
                max_depth = 25, min_child_weight = 1.1, alpha = 0.5, gamma =  0.4, n_estimators = 210)
xg_reg.fit(hash_train, y_train)
y_pred = xg_reg.predict(hash_test)

for i in range (0, len(y_pred)):
    if y_pred[i] >= 0.5:       
        y_pred[i] = 1 
    else:  
        y_pred[i]=0
        
accuracy = accuracy_score(y_pred,y_test)
print(accuracy)

## LSTM

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional, Concatenate, Flatten, Attention
from keras.models import Model,Sequential
from keras.callbacks import EarlyStopping

In [ ]:
max_words = 10000
max_len = 140

In [ ]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = Bidirectional(LSTM(256,return_sequences=True))(layer)
    layer = Bidirectional(LSTM(4))(layer)
    layer = Dense(64)(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(1)(layer)
    layer = Activation('sigmoid')(layer)

    model = Model(inputs=inputs,outputs=layer)
    return model
model = RNN()
model.summary()

### Con Features - 75% del set


In [ ]:
#Multiples parametros - Entrenar con 75% del set
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

X_train = tweets_metrics.iloc[:,4:]
X_train["text"] = tweets_metrics["text"]

Y_train = tweets_metrics.target
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_train = Y_train.reshape(-1,1)

X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size=0.25)

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train["text"])

sequences = tok.texts_to_sequences(X_train["text"])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test["text"])
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

features = StandardScaler()
X_train_features = features.fit_transform(X_train.iloc[:,:-1])
X_test_features = features.transform(X_test.iloc[:,:-1])

#Guarda el mejor
weight_path="Checkpoints/LSTM.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=1)
callbacks = [checkpoint, early_stopping]

#Ejecuta el fit
model = RNN()
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit([sequences_matrix,X_train_features],Y_train,batch_size=24,epochs=10,validation_split=0.2,callbacks=callbacks,verbose=1)

#Carga el mejor y evalua
model.load_weights(weight_path)
accr = model.evaluate([test_sequences_matrix,X_test_features],Y_test)

print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

### Sin Features - 75% del set


In [ ]:
#Sin features
#Preparar datos para test sin features
X_train = tweets_metrics.text
Y_train = tweets_metrics.target
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_train = Y_train.reshape(-1,1)

#Comentar para generar submit
X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size=0.25)

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

#Guarda el mejor
weight_path="Checkpoints/LSTM_No_Features.hdf5"
checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", mode="min", patience=1)
callbacks = [checkpoint, early_stopping]

#Ejecuta el fit
model = RNN()
model.compile(loss='binary_crossentropy',optimizer="adam",metrics=['accuracy'])
model.fit(sequences_matrix,Y_train,batch_size=71,epochs=10,validation_split=0.2,callbacks=callbacks,verbose=1)

#Carga el mejor y evalua
model.load_weights(weight_path)

#Comentar para generar submit - Sin features
accr = model.evaluate(test_sequences_matrix,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

## Logistic Regression

### CountVectorizer

In [ ]:
from sklearn.linear_model import LogisticRegression
x_train = tweets_metrics.text
y_train = tweets_metrics.target
x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.25)

model = LogisticRegression()

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word')
vectorizer.fit(x_train)
x_train = vectorizer.transform(x_train)
x_test  = vectorizer.transform(x_test)

model.fit(x_train, y_train)
score = model.score(x_test, y_test)

print("Presicion:", score)


### TF-IDF

In [ ]:
from sklearn.linear_model import LogisticRegression
x_train = tweets_metrics.text
y_train = tweets_metrics.target

x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.20)

model = LogisticRegression(solver="newton-cg")

from sklearn.feature_extraction.text import TfidfVectorizer
    
vectorizer = TfidfVectorizer(analyzer="word", smooth_idf = False)
vectorizer.fit(x_train)
x_train = vectorizer.transform(x_train)
x_test  = vectorizer.transform(x_test)

model.fit(x_train, y_train)
score = model.score(x_test, y_test)

print("Presicion:", score)

### Hashing Vectorizer

In [ ]:
from sklearn.linear_model import LogisticRegression
x_train = tweets_metrics.text
y_train = tweets_metrics.target

x_train,x_test,y_train,y_test = train_test_split(x_train,y_train,test_size=0.20)

model = LogisticRegression(solver="newton-cg")

from sklearn.feature_extraction.text import HashingVectorizer
    
vectorizer = HashingVectorizer(analyzer="word",n_features=60000)
vectorizer.fit(x_train)
x_train = vectorizer.transform(x_train)
x_test  = vectorizer.transform(x_test)

model.fit(x_train, y_train)
score = model.score(x_test, y_test)

print("Presicion:", score)

## SVM

In [ ]:
from sklearn import svm
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
X_train = tweets_metrics.text
Y_train = tweets_metrics.target
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)
Y_train = Y_train.reshape(-1,1)

#Comentar para generar submit
X_train,X_test,Y_train,Y_test = train_test_split(X_train,Y_train,test_size=0.25)

### CountVectorizer

In [ ]:
vectorizer = CountVectorizer(analyzer='word')
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test  = vectorizer.transform(X_test)

#Ejecuta el fit
model = svm.SVC(kernel='linear')
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

### TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(analyzer="word", smooth_idf = False)
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test  = vectorizer.transform(X_test)

#Ejecuta el fit
model = svm.SVC(kernel='linear')
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

### Hashing Vectorizer

In [ ]:
vectorizer = HashingVectorizer(analyzer="word",n_features=800000)
vectorizer.fit(X_train)
X_train = vectorizer.transform(X_train)
X_test  = vectorizer.transform(X_test)

#Ejecuta el fit
model = svm.SVC(kernel='linear')
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(Y_test, Y_pred))

## Light GBM

In [ ]:
d_train = lgb.Dataset(X_train, y_train)

params : {
    'learning_rate' : 0.02,
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'num_leaves' : 500,
    'max_depth' : 2,
    'max_bin': 1000
}

gbm = lgb.train(params, d_train, 10000)

## Tensorflow Hub 20 dim

In [ ]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(model, output_shape=[20], input_shape=[], 
                           dtype=tf.string, trainable=True)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(FULLY_CONNECTED, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(x_train,
                    y_train,
                    epochs=70,
                    batch_size=512,
                    validation_data=(x_test, y_test),
                    verbose=1)

## Tensorflow Hub 50 dim

In [ ]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", output_shape=[50],
                           input_shape=[], dtype=tf.string)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(FULLY_CONNECTED, activation='softmax'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)

history = model.fit(x_train,
                    y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_test, y_test),
                    verbose=1,
                    callbacks = [callback])

## Char CNN Conv1D

In [ ]:
from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
from keras.models import Sequential
from keras import layers

# Parametros
input_size = 1014
embedding_size = 71
fully_connected_layers = 1024
num_of_classes = 1
dropout_p = 0.5
optimizer = 'adam'
loss = 'binary_crossentropy'

embedding_layer = Embedding(embedding_size+1, 
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

inputs = Input(shape=(input_size,), name='input', dtype='int64')
x = embedding_layer(inputs)

model = Sequential()
model.add(embedding_layer)
model.add(layers.Conv1D(FILTERS, SIZE, activation='relu'))
model.add(layers.MaxPool1D(pool_size=4))
model.add(layers.Dense(FULLY_CONNECTED, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

# MAURO WIP

# Keras

### CNN

In [ ]:
from keras.models import Sequential
from keras import layers
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=2,
                    verbose=1,
                    #validation_data=(x_test, y_test),
                    batch_size=65)
#loss, accuracy = model.evaluate(x_train, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:

from keras.models import Sequential
from keras.layers import Dropout, Flatten
from keras import layers
#embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=True))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(12, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
from keras.callbacks import EarlyStopping
callback = EarlyStopping(monitor = 'val_loss', patience = 1)

history = model.fit(x_train, y_train,
                    epochs=15,
                    verbose=1,
                    #validation_data=(x_test, y_test),
                    validation_split=0.1,
                    batch_size=88,
                    callbacks = [callback])

### K fold CV

In [ ]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding='utf-8') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [ ]:
def create_model(num_filters, kernel_size, vocab_size, embedding_dim, maxlen):
    model = Sequential()
    model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=True))
    model.add(layers.Conv1D(num_filters, kernel_size, activation='relu'))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dropout, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras import layers
from keras.preprocessing.text import Tokenizer
callback = EarlyStopping(monitor = 'val_loss', patience = 1, verbose=1)

x_train = tweets_metrics['text'].values
y_train = tweets_metrics['target'].values

# Main settings
epochs = 15
embedding_dim = 100
maxlen = 140

# Train-test split
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1, random_state=1000)

# Tokenize words
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
#x_test = tokenizer.texts_to_sequences(x_test)

# Adding 1 because of reserved 0 index
vocab_size = len(tokenizer.word_index) + 1

# Pad sequences with zeros
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
#x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)

embedding_matrix = create_embedding_matrix('glove.twitter.27B.100d.txt',tokenizer.word_index, embedding_dim)

# Parameter grid for grid search
param_grid = dict(num_filters=[32, 128, 144],
                      kernel_size=[3, 5, 7],
                      vocab_size=[vocab_size],
                      embedding_dim=[embedding_dim],
                      maxlen=[maxlen],
                      batch_size = [45,65,76,88])

model = KerasClassifier(build_fn=create_model,
                            epochs=epochs, validation_split=0.1,
                            verbose=1)

grid = RandomizedSearchCV(estimator=model, param_distributions=param_grid,
                              cv=4, verbose=2, n_iter=5, n_jobs=1)

grid_result = grid.fit(x_train, y_train, callbacks=[callback])

# Evaluate testing set
#test_accuracy = grid.score(x_test, y_test)

# Save and evaluate results
s = ('Best Accuracy : {:.4f}\n{}\n\n\n')
output_string = s.format(
            grid_result.best_score_,
            grid_result.best_params_)
            
print(output_string)